# SPTAG python wrapper tutorial 

To end-to-end build a vector search online service, it contains two steps:
- Offline build SPTAG index for database vectors
- Online serve the index to support vector search requests from the clients

## Offline build SPTAG index

> Prepare input vectors and metadatas for SPTAG

In [1]:
import os
import shutil
import numpy as np

vector_number = 1000
vector_dimension = 100

# Randomly generate the database vectors. Currently SPTAG only support int8, int16 and float32 data type.
x = np.random.rand(vector_number, vector_dimension).astype(np.float32) 

# Prepare metadata for each vectors, separate them by '\n'. Currently SPTAG python wrapper only support '\n' as the separator
m = ''
for i in range(vector_number):
    m += str(i) + '\n'

> Build SPTAG index for database vectors **x**

In [2]:
import SPTAG

index = SPTAG.AnnIndex('BKT', 'Float', vector_dimension)

# Set the thread number to speed up the build procedure in parallel 
index.SetBuildParam("NumberOfThreads", '4', "Index")

# Set the distance type. Currently SPTAG only support Cosine and L2 distances. Here Cosine distance is not the Cosine similarity. The smaller Cosine distance it is, the better.
index.SetBuildParam("DistCalcMethod", 'L2', "Index") 

if (os.path.exists("sptag_index")):
    shutil.rmtree("sptag_index")
if index.BuildWithMetaData(x, m, vector_number, False, False):
    index.Save("sptag_index") # Save the index to the disk

os.listdir('sptag_index')

OSError: [WinError 145] The directory is not empty: 'sptag_index'

In [3]:
# Local index test on the vector search
index = SPTAG.AnnIndex.Load('sptag_index')

# prepare query vector
q = np.random.rand(vector_dimension).astype(np.float32)

result = index.SearchWithMetaData(q, 10) # Search k=3 nearest vectors for query vector q
print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas

[834, 350, 137, 220, 116, 411, 520, 585, 638, 788]
[10.045921325683594, 11.521989822387695, 11.940682411193848, 11.962193489074707, 12.010490417480469, 12.052820205688477, 12.099201202392578, 12.1024169921875, 12.24040699005127, 12.295062065124512]
[b'834\n', b'350\n', b'137\n', b'220\n', b'116\n', b'411\n', b'520\n', b'585\n', b'638\n', b'788\n']


> Build SPTAG index for database vector **x** with PQ quantization

In [4]:
import subprocess
import struct
import numpy as np

if (os.path.exists("quantizer.bin")):
    os.remove("quantizer.bin")
if (os.path.exists("quan_doc_vectors.bin")):
    os.remove("quan_doc_vectors.bin")
cmd = "Quantizer.exe -d %d -v Float -f DEFAULT -i sptag_index\\vectors.bin -o quan_doc_vectors.bin -oq quantizer.bin -qt PQQuantizer -qd %d -ts %d" % (vector_dimension, int(vector_dimension / 2), vector_number)
result = subprocess.run(cmd)
print (result) 
# For SPTAG index with quantization:
#     Cosine distance: norm -> true and use L2 for index build
#     L2 distance: norm -> false and use L2 for index build

f = open('quan_doc_vectors.bin', 'rb')
r = struct.unpack('i', f.read(4))[0]
c = struct.unpack('i', f.read(4))[0]
quan_x = np.frombuffer(f.read(r*c), dtype=np.uint8).reshape((r,c))
f.close()
print (quan_x.shape)

CompletedProcess(args='Quantizer.exe -d 100 -v Float -f DEFAULT -i sptag_index\\vectors.bin -o quan_doc_vectors.bin -oq quantizer.bin -qt PQQuantizer -qd 50 -ts 1000', returncode=0)
(1000, 50)


In [ ]:
import SPTAG

index = SPTAG.AnnIndex('BKT', 'Float', int(vector_dimension / 2))

# Set the thread number to speed up the build procedure in parallel 
index.SetBuildParam("NumberOfThreads", '4', "Index")

# Set the distance type. Currently SPTAG only support Cosine and L2 distances. Here Cosine distance is not the Cosine similarity. The smaller Cosine distance it is, the better.
index.SetBuildParam("DistCalcMethod", 'L2', "Index") 

if (os.path.exists("quan_sptag_index")):
    shutil.rmtree("quan_sptag_index")
if index.LoadQuantizer("quantizer.bin") and index.BuildWithMetaData(quan_x, m, vector_number, False, False):
    index.Save("quan_sptag_index") # Save the index to the disk

os.listdir('quan_sptag_index')

Error: Canceled future for execute_request message before replies were done

In [7]:
# Local index test on the vector search
index = SPTAG.AnnIndex.Load('quan_sptag_index')
index.SetQuantizerADC(True)
result = index.SearchWithMetaData(q, 12) # Search k=3 nearest vectors for query vector q
print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas

[67, 86, 56, 80, 64, 69, 72, 97, 78, 75, 95, 48]
[-1.3506038188934326, -1.3445258140563965, -1.340097427368164, -1.3317294120788574, -1.3258111476898193, -1.3229289054870605, -1.3226232528686523, -1.3196704387664795, -1.3155016899108887, -1.3154101371765137, -1.3148417472839355, -1.3078906536102295]
[b'67\n', b'86\n', b'56\n', b'80\n', b'64\n', b'69\n', b'72\n', b'97\n', b'78\n', b'75\n', b'95\n', b'48\n']


## Online serve the index

Start the vector search service on the host machine which listens for the client requests on the port 8000

> Write a server configuration file **service.ini** as follows:

```bash
[Service]
ListenAddr=0.0.0.0
ListenPort=8000
ThreadNumber=8
SocketThreadNumber=8

[QueryConfig]
DefaultMaxResultNumber=6
DefaultSeparator=|

[Index]
List=MyIndex

[Index_MyIndex]
IndexFolder=sptag_index
```

> Start the server on the host machine

```bash
Server.exe -m socket -c service.ini
```

It will print the follow messages:

```bash
Setting TreeFilePath with value tree.bin
Setting GraphFilePath with value graph.bin
Setting VectorFilePath with value vectors.bin
Setting DeleteVectorFilePath with value deletes.bin
Setting BKTNumber with value 1
Setting BKTKmeansK with value 32
Setting BKTLeafSize with value 8
Setting Samples with value 1000
Setting TPTNumber with value 32
Setting TPTLeafSize with value 2000
Setting NumTopDimensionTpTreeSplit with value 5
Setting NeighborhoodSize with value 32
Setting GraphNeighborhoodScale with value 2
Setting GraphCEFScale with value 2
Setting RefineIterations with value 2
Setting CEF with value 1000
Setting MaxCheckForRefineGraph with value 8192
Setting NumberOfThreads with value 4
Setting DistCalcMethod with value Cosine
Setting DeletePercentageForRefine with value 0.400000
Setting AddCountForRebuild with value 1000
Setting MaxCheck with value 8192
Setting ThresholdOfNumberOfContinuousNoBetterPropagation with value 3
Setting NumberOfInitialDynamicPivots with value 50
Setting NumberOfOtherDynamicPivots with value 4
Load Vector From sptag_index\vectors.bin
Load Vector (100, 10) Finish!
Load BKT From sptag_index\tree.bin
Load BKT (1,101) Finish!
Load Graph From sptag_index\graph.bin
Load Graph (100, 32) Finish!
Load DeleteID From sptag_index\deletes.bin
Load DeleteID (100, 1) Finish!
Start to listen 0.0.0.0:8000 ...
```

> Start python client to connect to the server and send vector search request.

In [4]:
import SPTAGClient
import time

# connect to the server
client = SPTAGClient.AnnClient('127.0.0.1', '8000')
while not client.IsConnected():
    time.sleep(1)
client.SetTimeoutMilliseconds(18000)

k = 3
vector_dimension = 10
# prepare query vector
q = np.random.rand(vector_dimension).astype(np.float32)

result = client.Search(q, k, 'Float', True) # AnnClient.Search(query_vector, knn, data_type, with_metadata)

print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas



[15, 35, 75]
[-1.094923973083496, -1.0913333892822266, -1.0795197486877441]
['15\n', '35\n', '75\n']
